# High-Resolution Daily Extreme Heat Days Calculator

Calculate extreme heat days using daily air temperature data at 1km resolution.

**Formula:** `Heat_Day = 1 if T_daily_max > max(T_abs, T_rel)`

Where:
- `T_rel = percentile_X{T_daily_max for calendar_day ± 5 days over baseline period}`
- **Dataset:** Daily Air Temperature (2003-2020, 1km resolution, global coverage)
- **Output:** GeoTIFF with extreme heat days per year per pixel

In [1]:
# Import libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
import io
import rasterio
from rasterio.crs import CRS
warnings.filterwarnings('ignore')

# Initialize Google Earth Engine
try:
    ee.Initialize(project='tl-cities')
    print("✅ Google Earth Engine initialized")
except Exception as e:
    print(f"❌ GEE initialization failed: {e}")

print("🌡️ High-Resolution Daily Extreme Heat Days Calculator Ready")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


✅ Google Earth Engine initialized
🌡️ High-Resolution Daily Extreme Heat Days Calculator Ready


## Load Daily Air Temperature Dataset and Define ROI

In [2]:
# =============================================================================
# ROI (Region of Interest) Selection & Management
# =============================================================================

print("🎯 Setting up ROI selection options...")

# Create ROI selection widgets
roi_method = widgets.RadioButtons(
    options=['Use Test Areas', 'Draw ROI on Map', 'Upload Reference Raster'],
    value='Use Test Areas',
    description='ROI Method:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Test area selector - Global options for daily air temperature data
test_area_selector = widgets.Dropdown(
    options=[
        ('Salvador, Brazil (5km x 5km)', 'salvador_small'),
        ('Phoenix, USA (10km x 10km)', 'phoenix_medium'),
        ('Madrid, Spain (5km x 5km)', 'madrid_small'),
        ('Lagos, Nigeria (8km x 8km)', 'lagos_medium'),
        ('Sydney, Australia (5km x 5km)', 'sydney_small')
    ],
    value='salvador_small',
    description='Test Area:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

file_upload = widgets.FileUpload(
    accept='.tif,.tiff,.geotiff',
    multiple=False,
    description='Upload GeoTIFF:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

roi_status = widgets.Output(layout=widgets.Layout(height='120px'))

# ROI management buttons
set_test_area_button = widgets.Button(
    description='🎯 Set Test Area',
    button_style='success',
    layout=widgets.Layout(width='140px')
)

set_roi_button = widgets.Button(
    description='📍 Set from Drawing',
    button_style='info',
    layout=widgets.Layout(width='140px')
)

show_roi_button = widgets.Button(
    description='👁️ Show ROI',
    button_style='warning',
    layout=widgets.Layout(width='100px')
)

clear_roi_button = widgets.Button(
    description='🗑️ Clear',
    button_style='',
    layout=widgets.Layout(width='80px')
)

# Global variables for ROI
analysis_geom = None
target_projection = None
target_scale = 1000  # Default 1km for daily air temperature data

def setup_roi_map():
    """Create initial map for ROI selection"""
    roi_map = geemap.Map(center=[20.0, 0.0], zoom=2)  # Global view
    roi_map.add_basemap('SATELLITE')
    
    # Add example locations for reference - Global cities
    example_cities = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point([-38.5014, -12.9714]), {'name': 'Salvador, Brazil'}),
        ee.Feature(ee.Geometry.Point([-112.0740, 33.4484]), {'name': 'Phoenix, USA'}),
        ee.Feature(ee.Geometry.Point([-3.7038, 40.4168]), {'name': 'Madrid, Spain'}),
        ee.Feature(ee.Geometry.Point([3.3792, 6.5244]), {'name': 'Lagos, Nigeria'}),
        ee.Feature(ee.Geometry.Point([151.2093, -33.8688]), {'name': 'Sydney, Australia'})
    ])
    
    roi_map.addLayer(example_cities, {'color': 'yellow'}, 'Example Cities')
    
    return roi_map

def get_test_area_geometry(area_code):
    """Get predefined test area geometries - Global locations"""
    if area_code == 'salvador_small':
        center_lon, center_lat = -38.5014, -12.9714
        buffer = 0.025  # ~2.5km
    elif area_code == 'phoenix_medium':
        center_lon, center_lat = -112.0740, 33.4484
        buffer = 0.05   # ~5km
    elif area_code == 'madrid_small':
        center_lon, center_lat = -3.7038, 40.4168
        buffer = 0.025  # ~2.5km
    elif area_code == 'lagos_medium':
        center_lon, center_lat = 3.3792, 6.5244
        buffer = 0.04   # ~4km
    elif area_code == 'sydney_small':
        center_lon, center_lat = 151.2093, -33.8688
        buffer = 0.025  # ~2.5km
    else:
        center_lon, center_lat = -38.5014, -12.9714
        buffer = 0.025
    
    return ee.Geometry.Rectangle([
        center_lon - buffer,
        center_lat - buffer, 
        center_lon + buffer,
        center_lat + buffer
    ])

def get_region_collection(geometry):
    """Get appropriate regional collection based on geometry location"""
    # Get centroid of geometry
    centroid = geometry.centroid()
    coords = centroid.coordinates().getInfo()
    lon, lat = coords[0], coords[1]
    
    # Determine region based on coordinates
    if -180 <= lon <= -30:  # Americas
        if lat >= 0:
            return 'projects/sat-io/open-datasets/global-daily-air-temp/north_america'
        else:
            return 'projects/sat-io/open-datasets/global-daily-air-temp/latin_america'
    elif -30 < lon <= 50:  # Europe/Africa
        if lat >= 20:
            return 'projects/sat-io/open-datasets/global-daily-air-temp/europe_asia'
        else:
            return 'projects/sat-io/open-datasets/global-daily-air-temp/africa'
    elif 50 < lon <= 180:  # Asia/Australia
        if lat >= -10:
            return 'projects/sat-io/open-datasets/global-daily-air-temp/europe_asia'
        else:
            return 'projects/sat-io/open-datasets/global-daily-air-temp/australia'
    else:
        # Default to europe_asia
        return 'projects/sat-io/open-datasets/global-daily-air-temp/europe_asia'

def handle_roi_method_change(change):
    """Handle ROI method selection"""
    with roi_status:
        clear_output(wait=True)
        
        if change['new'] == 'Use Test Areas':
            print("🎯 Test Areas Method Selected")
            print("Quick start with predefined areas around global cities.")
            test_area_selector.layout.display = 'block'
            set_test_area_button.layout.display = 'block'
            file_upload.layout.display = 'none'
            set_roi_button.layout.display = 'none'
            
        elif change['new'] == 'Draw ROI on Map':
            print("📍 Draw ROI Method Selected")
            print("Use the drawing tools on the map below to create custom areas.")
            test_area_selector.layout.display = 'none'
            set_test_area_button.layout.display = 'none'
            file_upload.layout.display = 'none'
            set_roi_button.layout.display = 'block'
            
        else:  # Upload Reference Raster
            print("📁 Upload Raster Method Selected")
            print("Upload a GeoTIFF to use its extent as the analysis area.")
            print("💡 Supported: .tif, .tiff files with spatial reference")
            test_area_selector.layout.display = 'none'
            set_test_area_button.layout.display = 'none'
            file_upload.layout.display = 'block'
            set_roi_button.layout.display = 'none'

def set_test_area():
    """Set predefined test area"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        try:
            area_code = test_area_selector.value
            analysis_geom = get_test_area_geometry(area_code)
            target_projection = 'EPSG:4326'
            target_scale = 1000  # 1km for daily air temperature data
            
            # Get area info
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            area_name = [opt[0] for opt in test_area_selector.options if opt[1] == area_code][0]
            
            # Determine which regional collection to use
            collection_id = get_region_collection(analysis_geom)
            region_name = collection_id.split('/')[-1].replace('_', ' ').title()
            
            print(f"✅ Test area set: {area_name}")
            print(f"   Area: {area_km2:.1f} km²")
            print(f"   Region: {region_name}")
            print(f"   Projection: {target_projection}")
            print(f"   Resolution: {target_scale}m")
            print("Ready for analysis!")
            
            # Auto-show on map
            show_roi_on_map(quiet=True)
            
        except Exception as e:
            print(f"❌ Error setting test area: {e}")

def process_uploaded_geotiff(file_content, filename):
    """
    Process uploaded GeoTIFF file and extract spatial information
    """
    try:
        # Read the file content using rasterio
        with rasterio.open(io.BytesIO(file_content)) as dataset:
            # Get spatial information
            bounds = dataset.bounds  # (left, bottom, right, top)
            crs = dataset.crs
            transform = dataset.transform
            width = dataset.width
            height = dataset.height
            
            # Convert bounds to WGS84 if needed
            if crs != CRS.from_epsg(4326):
                # Transform bounds to WGS84
                from rasterio.warp import transform_bounds
                wgs84_bounds = transform_bounds(crs, CRS.from_epsg(4326), *bounds)
            else:
                wgs84_bounds = bounds
            
            # Calculate resolution in degrees (approximate)
            pixel_size_x = abs(transform[0])
            pixel_size_y = abs(transform[4])
            
            # Convert to approximate meters (rough calculation at equator)
            approx_res_x_m = pixel_size_x * 111000  # degrees to meters
            approx_res_y_m = pixel_size_y * 111000
            approx_res_m = (approx_res_x_m + approx_res_y_m) / 2
            
            # Create Earth Engine geometry from bounds
            left, bottom, right, top = wgs84_bounds
            geometry = ee.Geometry.Rectangle([left, bottom, right, top])
            
            # Calculate area
            area_km2 = geometry.area().divide(1000000).getInfo()
            
            return {
                'geometry': geometry,
                'bounds': wgs84_bounds,
                'original_crs': str(crs),
                'target_crs': 'EPSG:4326',
                'resolution_m': int(approx_res_m),
                'area_km2': area_km2,
                'width': width,
                'height': height,
                'filename': filename
            }
            
    except Exception as e:
        raise Exception(f"Failed to process GeoTIFF: {str(e)}")

def handle_file_upload(change):
    """Handle uploaded raster file"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        if file_upload.value:
            try:
                # Get the uploaded file
                uploaded_file = file_upload.value[0]
                filename = uploaded_file['name']
                file_content = uploaded_file['content']
                
                print(f"🔄 Processing uploaded raster: {filename}")
                print("   Reading GeoTIFF spatial information...")
                
                # Process the uploaded GeoTIFF
                spatial_info = process_uploaded_geotiff(file_content, filename)
                
                # Set the analysis parameters
                analysis_geom = spatial_info['geometry']
                target_projection = spatial_info['target_crs']
                # FIXED: Keep consistent 1km resolution instead of using uploaded resolution
                target_scale = 1000  # Always use 1km for daily air temperature data consistency
                
                # Determine which regional collection to use
                collection_id = get_region_collection(analysis_geom)
                region_name = collection_id.split('/')[-1].replace('_', ' ').title()
                
                print(f"✅ GeoTIFF processed successfully!")
                print(f"   File: {filename}")
                print(f"   Original CRS: {spatial_info['original_crs']}")
                print(f"   Converted to: {spatial_info['target_crs']}")
                print(f"   Dimensions: {spatial_info['width']} × {spatial_info['height']} pixels")
                print(f"   Area: {spatial_info['area_km2']:.1f} km²")
                print(f"   Original resolution: {spatial_info['resolution_m']}m (derived from raster)")
                print(f"   Analysis resolution: {target_scale}m (fixed for daily air temp data)")
                print(f"   Region: {region_name}")
                print("Ready for analysis!")
                
                # Auto-show on map
                show_roi_on_map(quiet=True)
                
            except Exception as e:
                print(f"❌ Error processing uploaded file: {e}")
                print("\n💡 Troubleshooting:")
                print("• Ensure file is a valid GeoTIFF (.tif/.tiff)")
                print("• File must have spatial reference information (CRS)")
                print("• Try with a smaller file if processing fails")
                print("• Supported projections: any (will be converted to WGS84)")

def set_roi_from_drawing():
    """Get ROI from drawn geometry on map"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        try:
            # Try to get drawn geometry
            if hasattr(roi_map, 'draw_features') and roi_map.draw_features:
                drawn_feature = roi_map.draw_features[-1]
                geometry_data = drawn_feature['geometry']
                
                # Convert to EE geometry
                if geometry_data['type'] == 'Polygon':
                    coords = geometry_data['coordinates'][0]
                    analysis_geom = ee.Geometry.Polygon(coords)
                elif geometry_data['type'] == 'Rectangle':
                    coords = geometry_data['coordinates'][0]
                    analysis_geom = ee.Geometry.Rectangle(coords)
                else:
                    raise ValueError(f"Unsupported geometry type: {geometry_data['type']}")
                
                target_projection = 'EPSG:4326'
                target_scale = 1000
                
                # Get area info
                area_km2 = analysis_geom.area().divide(1000000).getInfo()
                collection_id = get_region_collection(analysis_geom)
                region_name = collection_id.split('/')[-1].replace('_', ' ').title()
                
                print(f"✅ ROI set from drawing")
                print(f"   Area: {area_km2:.1f} km²")
                print(f"   Region: {region_name}")
                print(f"   Projection: {target_projection}")
                print(f"   Resolution: {target_scale}m")
                print("Ready for analysis!")
                
                # Auto-show on analysis map
                show_roi_on_map(quiet=True)
                
            else:
                print("❌ No drawing found")
                print("Please draw a rectangle or polygon on the map first")
                
        except Exception as e:
            print(f"❌ Error setting ROI: {e}")
            print("Creating fallback test area...")
            
            # Fallback
            analysis_geom = ee.Geometry.Rectangle([-38.55, -12.95, -38.45, -12.90])
            target_projection = 'EPSG:4326' 
            target_scale = 1000
            
            print("✅ Using fallback test area")

def show_roi_on_map(quiet=False):
    """Display current ROI on the analysis map"""
    if not quiet:
        with roi_status:
            clear_output(wait=True)
    
    if analysis_geom is None:
        if not quiet:
            print("❌ No ROI set yet. Please set an ROI first.")
        return
        
    try:
        # Add ROI to the analysis map (m)
        roi_style = {
            'color': 'red',
            'fillColor': 'red',
            'fillOpacity': 0.2,
            'width': 3,
            'opacity': 1.0
        }
        m.addLayer(analysis_geom, roi_style, 'Current ROI')
        
        # Center map on ROI
        m.centerObject(analysis_geom, 10)
        
        if not quiet:
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            print(f"✅ ROI displayed on analysis map (Area: {area_km2:.1f} km²)")
        
    except Exception as e:
        if not quiet:
            print(f"❌ Error showing ROI: {e}")

def clear_roi():
    """Clear current ROI"""
    global analysis_geom, target_projection, target_scale
    
    with roi_status:
        clear_output(wait=True)
        
        analysis_geom = None
        target_projection = None
        target_scale = 1000
        
        print("✅ ROI cleared")
        print("Please select and set a new ROI for analysis")

# Connect event handlers
roi_method.observe(handle_roi_method_change, names='value')
file_upload.observe(handle_file_upload, names='value')
set_test_area_button.on_click(lambda b: set_test_area())
set_roi_button.on_click(lambda b: set_roi_from_drawing())
show_roi_button.on_click(lambda b: show_roi_on_map())
clear_roi_button.on_click(lambda b: clear_roi())

# Create ROI control interface
roi_buttons = widgets.HBox([
    set_test_area_button,
    set_roi_button,
    show_roi_button,
    clear_roi_button
])

roi_controls = widgets.VBox([
    widgets.HTML("<h4>🎯 Region of Interest (ROI) Selection</h4>"),
    roi_method,
    test_area_selector,
    file_upload,
    roi_buttons,
    roi_status
])

# Create and display ROI map
roi_map = setup_roi_map()

# Display interface
display(roi_controls)
display(roi_map)

# Initialize with default method
handle_roi_method_change({'new': 'Use Test Areas'})

print("✅ ROI selection interface ready!")
print("\n💡 Quick Start:")
print("1. Keep 'Salvador, Brazil (5km x 5km)' selected")
print("2. Click '🎯 Set Test Area'")
print("3. Scroll down to run analysis")

# Load daily air temperature data sources
print("\nLoading daily air temperature data sources...")
try:
    # Load regional collections - will be selected automatically based on ROI
    daily_temp_collections = {
        'north_america': ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/north_america"),
        'latin_america': ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/latin_america"),
        'europe_asia': ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/europe_asia"),
        'africa': ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/africa"),
        'australia': ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/australia")
    }
    print("✅ Daily air temperature collections loaded successfully")
    print("   Coverage: 2003-2020, 1km resolution, Global")
    print("   Variables: Daily Tmax and Tmin")
    
except Exception as e:
    print(f"❌ Daily air temperature loading failed: {e}")

print("\n📝 Note: Daily air temperature data provides true daily resolution")
print("   Perfect for daily extreme heat days analysis with no data gaps")

🎯 Setting up ROI selection options...


Map(center=[20.0, 0.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

✅ ROI selection interface ready!

💡 Quick Start:
1. Keep 'Salvador, Brazil (5km x 5km)' selected
2. Click '🎯 Set Test Area'
3. Scroll down to run analysis

Loading daily air temperature data sources...
✅ Daily air temperature collections loaded successfully
   Coverage: 2003-2020, 1km resolution, Global
   Variables: Daily Tmax and Tmin

📝 Note: Daily air temperature data provides true daily resolution
   Perfect for daily extreme heat days analysis with no data gaps


## Temperature Variable Selection & Heat Calculation Functions

In [3]:
def get_temperature_collection(region_geom, start_date, end_date, temp_type='tmax'):
    """
    Get daily air temperature collection (tmax or tmin) for the specified region and period
    Uses automatic regional collection selection and proper temperature scaling (divide by 10)
    """
    # Get appropriate regional collection based on ROI location
    collection_id = get_region_collection(region_geom)
    collection = ee.ImageCollection(collection_id)
    
    # Filter by date, bounds, and temperature type using prop_type metadata
    filtered_collection = collection.filterDate(start_date, end_date) \
                                  .filterBounds(region_geom) \
                                  .filter(ee.Filter.eq('prop_type', temp_type))
    
    # Apply temperature scaling (divide by 10) and convert to Celsius
    temp_collection = filtered_collection.map(lambda img: 
        img.select('b1')
          .divide(10)  # Scale from raw values to Celsius
          .rename('temperature').clip(region_geom)
    )
    
    return temp_collection

def calculate_simple_extreme_heat_days_daily(region_geom, year, temp_abs_threshold=35, 
                                           percentile_threshold=95, 
                                           baseline_start='2010-01-01', baseline_end='2019-12-31'):
    """
    FIXED: Simplified extreme heat days calculation using daily air temperature data (tmax)
    Uses single percentile across entire baseline period (no seasonal acclimatization)
    NOTE: For proper acclimatization analysis, use Full Percentiles mode instead
    """
    print(f"   Calculating extreme heat days for {year} using Daily Air Temperature (Simplified Mode)...")
    print(f"   Temperature variable: Daily Maximum (Tmax)")
    print(f"   ⚠️  Simplified mode uses overall percentile (no seasonal acclimatization)")
    
    # Get baseline tmax collection
    baseline_collection = get_temperature_collection(region_geom, baseline_start, baseline_end, 'tmax')
    
    baseline_count = baseline_collection.size().getInfo()
    print(f"   Baseline tmax images found: {baseline_count}")
    
    # Calculate simple percentile across all days in baseline period
    print(f"   Computing single {percentile_threshold}th percentile across baseline period...")
    temp_percentile = baseline_collection.select('temperature').reduce(ee.Reducer.percentile([percentile_threshold])).clip(region_geom)
    
    # Get year tmax collection
    year_collection = get_temperature_collection(region_geom, f'{year}-01-01', f'{year}-12-31', 'tmax')
    
    year_count = year_collection.size().getInfo()
    print(f"   Target year tmax images found: {year_count}")
    
    # Create absolute threshold image
    temp_abs = ee.Image.constant(temp_abs_threshold).rename('temp_abs').clip(region_geom)
    
    # Use max of absolute threshold and calculated percentile
    threshold = temp_abs.max(temp_percentile).rename('threshold').clip(region_geom)
    
    print(f"   🔍 Checking threshold values...")
    # Debug: Check threshold values
    try:
        threshold_stats = threshold.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=region_geom,
            scale=2000,
            maxPixels=1e6,
            bestEffort=True
        ).getInfo()
        
        thresh_min = threshold_stats.get('threshold_min', 'N/A')
        thresh_max = threshold_stats.get('threshold_max', 'N/A')
        print(f"   Actual threshold used: {thresh_min:.1f}°C to {thresh_max:.1f}°C")
        
        # Quick heat day estimate
        sample_temps = year_collection.first().select('temperature')
        sample_stats = sample_temps.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=region_geom,
            scale=2000,
            maxPixels=1e6,
            bestEffort=True
        ).getInfo()
        
        temp_min = sample_stats.get('temperature_min', 'N/A')
        temp_max = sample_stats.get('temperature_max', 'N/A')
        print(f"   Sample day temps: {temp_min:.1f}°C to {temp_max:.1f}°C")
        
        if isinstance(thresh_min, (int, float)) and isinstance(temp_max, (int, float)):
            if temp_max > thresh_min:
                print(f"   ✅ Some days should exceed threshold")
            else:
                print(f"   ⚠️ Daily temps may rarely exceed threshold - this explains low heat days")
                
    except Exception as e:
        print(f"   Could not retrieve threshold stats: {e}")
    
    def classify_heat_day(img):
        """
        Classify each day as extreme heat day or not
        """
        temp_daily = img.select('temperature')
        # Create binary image: 1 where temp > threshold, 0 otherwise
        heat_day = temp_daily.gt(threshold).rename('heat_day').byte()
        
        # Ensure proper clipping and data type
        return heat_day.clip(region_geom).set({
            'system:time_start': img.get('system:time_start'),
            'date': img.get('system:time_start')
        })
    
    # Apply classification to all days
    print(f"   Classifying {year_count} days as heat/non-heat days...")
    heat_days_collection = year_collection.map(classify_heat_day)
    
    # FIXED: Ensure proper image collection reduction
    print(f"   Summing heat days across all {year_count} days...")
    
    # Direct sum of heat day binary images
    total_extreme_heat_days = heat_days_collection.select('heat_day').sum().rename('extreme_heat_days').clip(region_geom)
    
    # Convert to appropriate data type
    result = total_extreme_heat_days.toUint16().clip(region_geom)
    
    print(f"   Heat days calculation complete!")
    return result

def calculate_acclimatization_percentiles_daily(region_geom, percentile_value=95, 
                                               baseline_start='2010-01-01', baseline_end='2019-12-31',
                                               window_days=15):
    """
    Calculate seasonally-adjusted temperature percentiles for local acclimatization analysis
    
    For each calendar day, calculates percentiles within a moving window (±15-30 days) 
    across multiple baseline years. This accounts for:
    - Seasonal temperature variation (summer vs winter normals)  
    - Local climate acclimatization (what's extreme varies by location)
    - Population heat adaptation (what locals consider "unusually hot")
    
    Args:
        region_geom: ROI geometry
        percentile_value: Percentile to calculate (90-99th)
        baseline_start/end: Multi-year baseline period
        window_days: Days around each calendar day (±15 to ±30 recommended)
    
    Returns:
        Image with 365 bands of seasonally-adjusted percentile thresholds
    """
    print(f"   Calculating {percentile_value}th percentile for local acclimatization analysis...")
    print(f"   Using ±{window_days} day window for seasonal adjustment")
    print("   This accounts for local climate acclimatization patterns")
    print("   ⚠️  This may use significant memory for long baseline periods")
    
    # Get baseline period tmax collection
    baseline_collection = get_temperature_collection(region_geom, baseline_start, baseline_end, 'tmax')
    
    # Add day of year property
    baseline_temp = baseline_collection.map(lambda img: 
        img.select('temperature').clip(region_geom)
        .set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year'))
    )
    
    def calculate_acclimatization_percentile(doy):
        """
        Calculate seasonally-adjusted percentile for specific calendar day
        Uses moving window approach for local acclimatization
        """
        doy = ee.Number(doy)
        
        # Create wider window around target day (±15-30 days recommended for acclimatization)
        start_doy = doy.subtract(window_days)
        end_doy = doy.add(window_days)
        
        # Handle year wrapping (e.g., December 31 ± 15 days includes January dates)
        normal_filter = ee.Filter.And(
            ee.Filter.gte('doy', start_doy),
            ee.Filter.lte('doy', end_doy)
        )
        
        # Handle case where window wraps around year end/start
        wrap_filter = ee.Filter.Or(
            ee.Filter.gte('doy', start_doy.add(365)),  # End of year
            ee.Filter.lte('doy', end_doy.subtract(365))  # Start of year
        )
        
        day_filter = ee.Algorithms.If(
            start_doy.lt(1).Or(end_doy.gt(365)),
            wrap_filter,
            normal_filter
        )
        
        # Filter images for this seasonal window across all baseline years
        seasonal_images = baseline_temp.filter(day_filter)
        
        # Calculate percentile for this seasonal window (accounts for local acclimatization)
        percentile_img = seasonal_images.select('temperature').reduce(
            ee.Reducer.percentile([percentile_value])
        ).clip(region_geom)
        
        # Create band name
        band_name = ee.String('accl_p').cat(ee.Number(percentile_value).format()).cat('_doy_').cat(doy.format('%03d'))
        
        return percentile_img.rename([band_name])
    
    # Calculate acclimatization percentiles for all days of year (1-365)
    doy_list = ee.List.sequence(1, 365)
    percentile_images = doy_list.map(calculate_acclimatization_percentile)
    
    # Convert to multi-band image (365 bands of seasonally-adjusted thresholds)
    acclimatization_stack = ee.Image(ee.ImageCollection(percentile_images).toBands()).clip(region_geom)
    
    return acclimatization_stack

def calculate_extreme_heat_days_daily(region_geom, year, temp_abs_threshold=35, 
                                     percentile_threshold=95, baseline_start='2010-01-01', 
                                     baseline_end='2019-12-31', window_days=15):
    """
    Full extreme heat days calculation with local acclimatization analysis
    
    Uses seasonally-adjusted percentiles that account for:
    - Local climate acclimatization (what's normal varies by location)  
    - Seasonal heat adaptation (summer vs winter expectations)
    - Population heat tolerance (what locals consider extreme)
    
    Formula: Heat_Day = 1 if temp_daily_max > max(temp_abs, seasonal_percentile)
    where seasonal_percentile accounts for local acclimatization patterns
    
    WARNING: This is memory intensive due to day-specific percentile calculations
    """
    print(f"   Calculating extreme heat days for {year} using Local Acclimatization Analysis...")
    print(f"   Using ±{window_days} day seasonal window for acclimatization")
    print("   This accounts for local climate and seasonal heat adaptation")
    print("   ⚠️  This may use significant memory - consider simplified mode for testing")
    
    # Get year tmax collection
    year_collection = get_temperature_collection(region_geom, f'{year}-01-01', f'{year}-12-31', 'tmax')
    
    # Add day of year property
    year_temp = year_collection.map(lambda img: 
        img.select('temperature').clip(region_geom)
        .set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year'))
        .set('system:time_start', img.get('system:time_start'))
    )
    
    # Calculate seasonally-adjusted thresholds for acclimatization analysis
    print(f"     Computing {percentile_threshold}th percentile with ±{window_days} day seasonal windows...")
    acclimatization_stack = calculate_acclimatization_percentiles_daily(
        region_geom, percentile_threshold, baseline_start, baseline_end, window_days
    )
    
    # Create absolute threshold image
    temp_abs = ee.Image.constant(temp_abs_threshold).rename('temp_abs').clip(region_geom)
    
    def classify_heat_day_acclimatized(img):
        """
        Classify each day using seasonally-adjusted threshold for local acclimatization
        """
        doy = ee.Number(img.get('doy'))
        
        # Get the seasonally-adjusted percentile threshold for this calendar day
        band_name = ee.String('accl_p').cat(ee.Number(percentile_threshold).format()).cat('_doy_').cat(doy.format('%03d'))
        
        # Get acclimatization threshold for this day, with fallback
        seasonal_threshold = ee.Algorithms.If(
            acclimatization_stack.bandNames().contains(band_name),
            acclimatization_stack.select([band_name]),
            ee.Image.constant(temp_abs_threshold).clip(region_geom)  # Fallback to absolute threshold
        )
        seasonal_threshold = ee.Image(seasonal_threshold).rename('seasonal_threshold').clip(region_geom)
        
        # Calculate max(temp_abs, seasonal_threshold) - accounts for both absolute danger and local acclimatization
        final_threshold = temp_abs.max(seasonal_threshold).rename('threshold').clip(region_geom)
        
        # Temperature daily maximum
        temp_daily_max = img.select('temperature')
        
        # Heat_Day = 1 if temp_daily_max > acclimatization-adjusted threshold
        heat_day = temp_daily_max.gt(final_threshold).rename('heat_day').byte()
        
        return heat_day.clip(region_geom).set({
            'system:time_start': img.get('system:time_start'),
            'date': img.get('system:time_start')
        })
    
    # Apply acclimatization-based classification to all days
    heat_days_collection = year_temp.map(classify_heat_day_acclimatized)
    
    # Sum to get total extreme heat days (accounting for local acclimatization)
    print(f"   Summing acclimatization-based heat days...")
    total_extreme_heat_days = heat_days_collection.select('heat_day').sum().rename('extreme_heat_days').clip(region_geom)
    
    # Convert to appropriate data type
    result = total_extreme_heat_days.toUint16().clip(region_geom)
    
    return result

def align_to_target_grid(image, region_geom, target_crs='EPSG:4326', target_scale=1000):
    """
    FIXED: Align image to target grid by resampling - removed invalid 'nearest' method
    """
    print(f"   Reprojecting to {target_crs} at {target_scale}m resolution...")
    
    # First ensure image is properly clipped to exact ROI
    clipped_image = image.clip(region_geom)
    
    # Use bilinear resampling (valid method in GEE)
    aligned_image = clipped_image.resample('bilinear').reproject(
        crs=target_crs,
        scale=target_scale
    )
    
    # Ensure final result is properly bounded and has correct data type
    final_result = aligned_image.clip(region_geom).toUint16()
    
    return final_result

def parse_baseline_period(period_string):
    """Parse baseline period string to start/end dates"""
    if '2010-2019' in period_string:
        return '2010-01-01', '2019-12-31'
    elif '2005-2019' in period_string:
        return '2005-01-01', '2019-12-31'
    elif '2003-2019' in period_string:
        return '2003-01-01', '2019-12-31'
    else:
        return '2010-01-01', '2019-12-31'  # Default

print("✅ Daily air temperature extreme heat days calculation functions ready")
print("   🔧 FIXED: Removed invalid 'nearest' resampling method")
print("   🔧 ADDED: Enhanced threshold diagnostics")
print("   with proper local acclimatization methodology (±15 day seasonal windows)")

✅ Daily air temperature extreme heat days calculation functions ready
   🔧 FIXED: Removed invalid 'nearest' resampling method
   🔧 ADDED: Enhanced threshold diagnostics
   with proper local acclimatization methodology (±15 day seasonal windows)


## Visualization and Export Functions

In [4]:
def create_heat_visualization():
    """
    Create visualization parameters for extreme heat days
    """
    # Heat-focused color palette (yellow to red)
    heat_palette = ['#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', 
                   '#fc4e2a', '#e31a1c', '#bd0026', '#800026']
    
    return {
        'palette': heat_palette,
        'min': 0,
        'max': 100  # Days per year - will be adjusted dynamically
    }

def export_to_drive(image, region_geom, filename, target_crs='EPSG:4326', target_scale=1000, folder='GEE_Exports'):
    """
    Export image to Google Drive as GeoTIFF
    """
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        folder=folder,
        fileNamePrefix=filename,
        region=region_geom,
        scale=target_scale,
        crs=target_crs,
        maxPixels=1e9,
        fileFormat='GeoTIFF'
    )
    
    task.start()
    return task

def display_on_map(m, image, region_geom, layer_name, show_region=True):
    """
    Display results on interactive map with improved visualization for uniform/low-range data
    """
    try:
        # Use larger scale for stats calculation to reduce memory usage
        stats_scale = max(2000, target_scale * 2)  # At least 2km resolution for stats
        
        # Get statistics for visualization scaling
        stats = image.reduceRegion(
            reducer=ee.Reducer.minMax().combine(ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True), sharedInputs=True),
            geometry=region_geom,
            scale=stats_scale,
            maxPixels=1e6,  # Reduced pixel limit
            bestEffort=True
        )
        
        stats_info = stats.getInfo()
        min_val = stats_info.get('extreme_heat_days_min', 0) or 0
        max_val = stats_info.get('extreme_heat_days_max', 50) or 50
        mean_val = stats_info.get('extreme_heat_days_mean', 25) or 25
        std_val = stats_info.get('extreme_heat_days_stdDev', 0) or 0
        
        print(f"Data range: {min_val:.1f} to {max_val:.1f} extreme heat days per year")
        print(f"Mean: {mean_val:.1f} days, Std dev: {std_val:.2f} days")
        
        # Create visualization parameters with improved range handling
        vis_params = create_heat_visualization()
        
        # Handle different data range scenarios
        data_range = max_val - min_val
        
        if data_range <= 1:
            # Very uniform data (like 4 to 4 days) - stretch the visualization
            print("⚠️  Very uniform data detected - using enhanced contrast visualization")
            vis_params['min'] = max(0, min_val - 0.5)  # Slightly below min
            vis_params['max'] = min_val + 2  # Slightly above to show any variation
            
        elif data_range <= 10:
            # Low variation (0-10 days) - use tight range
            print("🔍 Low variation data - using tight range visualization")
            vis_params['min'] = min_val
            vis_params['max'] = max(min_val + 5, max_val + 1)  # Ensure some range
            
        else:
            # Normal range - use actual min/max
            vis_params['min'] = min_val
            vis_params['max'] = max_val
        
        # Use higher contrast palette for uniform data
        if std_val < 1:
            print("🎨 Using high contrast palette for low variation data")
            # More contrasting colors for subtle differences
            vis_params['palette'] = ['#000080', '#0000FF', '#0080FF', '#00FFFF', '#80FF80', 
                                   '#FFFF00', '#FF8000', '#FF0000', '#800000']
        
        print(f"Visualization range: {vis_params['min']:.1f} to {vis_params['max']:.1f}")
        
        # Add layer to map
        m.addLayer(image, vis_params, layer_name)
        
        # Add region boundary
        if show_region:
            region_style = {
                'color': 'blue',
                'fillColor': '00000000',
                'width': 2,
                'opacity': 1.0
            }
            m.addLayer(region_geom, region_style, 'Analysis ROI')
        
        # Center on region
        m.centerObject(region_geom, 12)  # Higher zoom for better visibility
        
        return min_val, max_val
        
    except Exception as e:
        print(f"⚠️ Could not calculate stats for visualization: {e}")
        print("Using default visualization parameters...")
        
        # Use default parameters if stats fail
        vis_params = create_heat_visualization()
        vis_params['min'] = 0
        vis_params['max'] = 20  # Lower max for better visibility of low values
        
        # Add layer to map
        m.addLayer(image, vis_params, layer_name)
        
        # Add region boundary
        if show_region:
            region_style = {
                'color': 'blue',
                'fillColor': '00000000',
                'width': 2,
                'opacity': 1.0
            }
            m.addLayer(region_geom, region_style, 'Analysis ROI')
        
        # Center on region
        m.centerObject(region_geom, 12)
        
        return 0, 20

print("✅ Visualization and export functions ready")
print("   🎨 Enhanced visualization for uniform/low-range data")

✅ Visualization and export functions ready
   🎨 Enhanced visualization for uniform/low-range data


## Interactive Analysis Interface

In [5]:
# Create interactive map
m = geemap.Map(center=[20.0, 0.0], zoom=2)  # Global view for daily air temperature data

# Create control widgets
year_selector = widgets.Dropdown(
    options=list(range(2005, 2021)),  # Daily air temp range: 2003-2020
    value=2020,
    description='Analysis Year:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='200px')
)

# Simplified - only tmax since this is for extreme heat days
temp_variable_selector = widgets.Dropdown(
    options=[
        ('Daily Maximum Temperature (Tmax)', 'tmax'),
        ('Daily Minimum Temperature (Tmin)', 'tmin')
    ],
    value='tmax',
    description='Temperature Variable:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='350px')
)

# Add simplified mode option
analysis_mode = widgets.RadioButtons(
    options=['Simplified (Fast)', 'Full Percentiles (Slow)'],
    value='Simplified (Fast)',
    description='Analysis Mode:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='250px')
)

baseline_period = widgets.Dropdown(
    options=['2010-2019 (10 years)', '2005-2019 (15 years)', '2003-2019 (17 years)'],
    value='2010-2019 (10 years)',
    description='Baseline Period:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='250px')
)

temp_abs_slider = widgets.FloatSlider(
    value=30.0,  # Better default for tropical locations
    min=25.0,
    max=50.0,
    step=1.0,
    description='T_abs (°C):',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

percentile_slider = widgets.IntSlider(
    value=85,  # Better default for tropical locations
    min=75,
    max=99,
    step=1,
    description='Percentile:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

calculate_button = widgets.Button(
    description='🔥 Calculate Heat Days',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='35px')
)

export_button = widgets.Button(
    description='📁 Export to Drive',
    button_style='success',
    layout=widgets.Layout(width='200px', height='35px'),
    disabled=True
)

status_output = widgets.Output(layout=widgets.Layout(height='200px'))

# Global variables
current_result = None
export_task = None

print("✅ User interface widgets created with daily air temperature options")

✅ User interface widgets created with daily air temperature options


## Main Analysis Functions

In [6]:
# Import required libraries for temperature trend analysis
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import numpy as np

def create_temperature_trend_analysis():
    """
    Create comprehensive temperature trend analysis for the ROI
    - Chart 1: Long-term trends (2003-2020) showing how max temperature varies over time
    - Chart 2: Daily temperature distribution for selected year with percentile and threshold lines
    """
    global analysis_geom
    
    if analysis_geom is None:
        print("❌ Please set an ROI first!")
        return
    
    print("📊 Creating temperature trend analysis for your ROI...")
    print("   This may take a few minutes to process the full time series...")
    
    year = year_selector.value
    temp_abs = temp_abs_slider.value
    percentile = percentile_slider.value
    
    try:
        # ============================================================================
        # CHART 1: LONG-TERM TEMPERATURE TRENDS (2003-2020)
        # ============================================================================
        print("   📈 Generating long-term temperature trends (2003-2020)...")
        
        # Get full temperature time series for trends
        full_collection = get_temperature_collection(
            analysis_geom, '2003-01-01', '2020-12-31', 'tmax'
        )
        
        # Sample every 30 days for manageable processing
        dates = []
        start_date = datetime(2003, 1, 1)
        end_date = datetime(2020, 12, 31)
        current_date = start_date
        
        while current_date <= end_date:
            dates.append(current_date.strftime('%Y-%m-%d'))
            current_date += timedelta(days=30)  # Sample every 30 days
        
        print(f"   Sampling {len(dates)} dates for trend analysis...")
        
        # Get temperature statistics for each sample date
        trend_data = {'dates': [], 'mean_temp': [], 'max_temp': [], 'min_temp': [], 'p95_temp': []}
        
        for i, date_str in enumerate(dates[:50]):  # Limit to first 50 samples for speed
            try:
                # Get single day collection
                daily_collection = full_collection.filterDate(date_str, 
                    (datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'))
                
                if daily_collection.size().getInfo() > 0:
                    daily_image = daily_collection.first()
                    
                    # Calculate statistics across ROI
                    stats = daily_image.select('temperature').reduceRegion(
                        reducer=ee.Reducer.minMax().combine(
                            ee.Reducer.mean().combine(
                                ee.Reducer.percentile([95]), sharedInputs=True
                            ), sharedInputs=True
                        ),
                        geometry=analysis_geom,
                        scale=2000,
                        maxPixels=1e6,
                        bestEffort=True
                    ).getInfo()
                    
                    trend_data['dates'].append(datetime.strptime(date_str, '%Y-%m-%d'))
                    trend_data['mean_temp'].append(stats.get('temperature_mean', None))
                    trend_data['max_temp'].append(stats.get('temperature_max', None))
                    trend_data['min_temp'].append(stats.get('temperature_min', None))
                    trend_data['p95_temp'].append(stats.get('temperature_p95', None))
                    
                if (i + 1) % 10 == 0:
                    print(f"   Processed {i + 1}/{len(dates[:50])} trend samples...")
                    
            except Exception as e:
                print(f"   Skipping date {date_str}: {e}")
                continue
        
        # ============================================================================
        # CHART 2: DAILY TEMPERATURE DISTRIBUTION FOR SELECTED YEAR
        # ============================================================================
        print(f"   📊 Generating daily temperature distribution for {year}...")
        
        # Get full year collection
        year_collection = get_temperature_collection(
            analysis_geom, f'{year}-01-01', f'{year}-12-31', 'tmax'
        )
        
        year_count = year_collection.size().getInfo()
        print(f"   Processing {year_count} days for {year}...")
        
        # Sample every 5 days for detailed year analysis
        year_data = {'day_of_year': [], 'temp_min': [], 'temp_max': [], 
                    'temp_mean': [], 'temp_p95': [], 'temp_p05': []}
        
        # Create list of sample dates (every 5 days)
        sample_dates = []
        start_year = datetime(year, 1, 1)
        for day_offset in range(0, 365, 5):  # Every 5 days
            sample_date = start_year + timedelta(days=day_offset)
            if sample_date.year == year:
                sample_dates.append(sample_date)
        
        for i, sample_date in enumerate(sample_dates):
            try:
                date_str = sample_date.strftime('%Y-%m-%d')
                next_date_str = (sample_date + timedelta(days=1)).strftime('%Y-%m-%d')
                
                # Get single day
                daily_collection = year_collection.filterDate(date_str, next_date_str)
                
                if daily_collection.size().getInfo() > 0:
                    daily_image = daily_collection.first()
                    
                    # Calculate spatial statistics for this day
                    stats = daily_image.select('temperature').reduceRegion(
                        reducer=ee.Reducer.minMax().combine(
                            ee.Reducer.mean().combine(
                                ee.Reducer.percentile([5, 95]), sharedInputs=True
                            ), sharedInputs=True
                        ),
                        geometry=analysis_geom,
                        scale=1000,  # Use full resolution for daily analysis
                        maxPixels=1e6,
                        bestEffort=True
                    ).getInfo()
                    
                    year_data['day_of_year'].append(sample_date.timetuple().tm_yday)
                    year_data['temp_min'].append(stats.get('temperature_min', None))
                    year_data['temp_max'].append(stats.get('temperature_max', None))
                    year_data['temp_mean'].append(stats.get('temperature_mean', None))
                    year_data['temp_p95'].append(stats.get('temperature_p95', None))
                    year_data['temp_p05'].append(stats.get('temperature_p05', None))
                    
                if (i + 1) % 10 == 0:
                    print(f"   Processed {i + 1}/{len(sample_dates)} daily samples...")
                    
            except Exception as e:
                print(f"   Skipping {sample_date}: {e}")
                continue
        
        # ============================================================================
        # Calculate baseline 95th percentile for comparison
        # ============================================================================
        print("   📊 Calculating baseline 95th percentile...")
        
        baseline_start, baseline_end = parse_baseline_period(baseline_period.value)
        baseline_collection = get_temperature_collection(
            analysis_geom, baseline_start, baseline_end, 'tmax'
        )
        
        baseline_p95 = baseline_collection.select('temperature').reduce(
            ee.Reducer.percentile([percentile])
        ).clip(analysis_geom)
        
        p95_stats = baseline_p95.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=analysis_geom,
            scale=2000,
            maxPixels=1e6,
            bestEffort=True
        ).getInfo()
        
        baseline_p95_value = p95_stats.get(f'temperature_p{percentile}', 35.0)
        
        # ============================================================================
        # CREATE PLOTS
        # ============================================================================
        print("   🎨 Creating visualizations...")
        
        # Create figure with subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
        
        # ============================================================================
        # PLOT 1: LONG-TERM TRENDS
        # ============================================================================
        if len(trend_data['dates']) > 0:
            # Remove None values
            valid_trend = [(d, mn, mx, avg, p95) for d, mn, mx, avg, p95 in 
                          zip(trend_data['dates'], trend_data['min_temp'], 
                              trend_data['max_temp'], trend_data['mean_temp'], 
                              trend_data['p95_temp']) 
                          if all(v is not None for v in [mn, mx, avg, p95])]
            
            if valid_trend:
                dates, mins, maxs, avgs, p95s = zip(*valid_trend)
                
                # Plot temperature ranges
                ax1.fill_between(dates, mins, maxs, alpha=0.3, color='lightblue', label='Min-Max Range')
                ax1.plot(dates, avgs, 'b-', linewidth=2, label='Mean Temperature')
                ax1.plot(dates, p95s, 'r--', linewidth=2, label='95th Percentile')
                
                # Add horizontal line for user threshold
                ax1.axhline(y=temp_abs, color='orange', linestyle='-', linewidth=2, 
                           label=f'User Threshold ({temp_abs}°C)')
                
                ax1.set_title(f'Temperature Trends Over Time (2003-2020)\nROI Area: ~{analysis_geom.area().divide(1000000).getInfo():.1f} km²', 
                             fontsize=14, fontweight='bold')
                ax1.set_xlabel('Year')
                ax1.set_ylabel('Temperature (°C)')
                ax1.legend()
                ax1.grid(True, alpha=0.3)
                
                # Add trend line for mean temperature
                import numpy as np
                x_numeric = [(d - dates[0]).days for d in dates]
                z = np.polyfit(x_numeric, avgs, 1)
                p = np.poly1d(z)
                ax1.plot(dates, p(x_numeric), "g--", alpha=0.8, linewidth=1, 
                        label=f'Trend: {z[0]*365:.3f}°C/year')
                ax1.legend()
        
        # ============================================================================
        # PLOT 2: DAILY DISTRIBUTION FOR SELECTED YEAR
        # ============================================================================
        if len(year_data['day_of_year']) > 0:
            # Remove None values
            valid_daily = [(doy, mn, mx, avg, p95, p05) for doy, mn, mx, avg, p95, p05 in 
                          zip(year_data['day_of_year'], year_data['temp_min'], 
                              year_data['temp_max'], year_data['temp_mean'],
                              year_data['temp_p95'], year_data['temp_p05']) 
                          if all(v is not None for v in [mn, mx, avg, p95, p05])]
            
            if valid_daily:
                days, mins, maxs, avgs, p95s, p05s = zip(*valid_daily)
                
                # Create temperature range visualization
                ax2.fill_between(days, p05s, p95s, alpha=0.4, color='lightcoral', 
                               label='5th-95th Percentile Range')
                ax2.fill_between(days, mins, maxs, alpha=0.2, color='lightblue', 
                               label='Min-Max Range (across pixels)')
                ax2.plot(days, avgs, 'b-', linewidth=2, label='Daily Mean')
                
                # Add horizontal lines for thresholds
                ax2.axhline(y=baseline_p95_value, color='red', linestyle='--', linewidth=2, 
                           label=f'Baseline {percentile}th Percentile ({baseline_p95_value:.1f}°C)')
                ax2.axhline(y=temp_abs, color='orange', linestyle='-', linewidth=2, 
                           label=f'User Threshold ({temp_abs}°C)')
                
                # Count days above thresholds
                days_above_p95 = sum(1 for avg in avgs if avg > baseline_p95_value)
                days_above_user = sum(1 for avg in avgs if avg > temp_abs)
                
                ax2.set_title(f'Daily Temperature Distribution - {year}\n'
                             f'Days above {percentile}th percentile: {days_above_p95}/{len(days)} | '
                             f'Days above user threshold: {days_above_user}/{len(days)}', 
                             fontsize=14, fontweight='bold')
                ax2.set_xlabel('Day of Year')
                ax2.set_ylabel('Temperature (°C)')
                ax2.legend()
                ax2.grid(True, alpha=0.3)
                
                # Add month labels
                month_starts = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
                month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                ax2.set_xticks(month_starts)
                ax2.set_xticklabels(month_names)
        
        plt.tight_layout()
        plt.show()
        
        # ============================================================================
        # SUMMARY STATISTICS
        # ============================================================================
        print("\n📊 Temperature Analysis Summary:")
        print(f"   ROI: Salvador area ({analysis_geom.area().divide(1000000).getInfo():.1f} km²)")
        print(f"   Analysis year: {year}")
        print(f"   Baseline period: {baseline_start} to {baseline_end}")
        print(f"   Baseline {percentile}th percentile: {baseline_p95_value:.1f}°C")
        print(f"   User threshold: {temp_abs}°C")
        
        if valid_daily:
            avg_temp = sum(avgs) / len(avgs)
            print(f"   Average temperature in {year}: {avg_temp:.1f}°C")
            print(f"   Temperature range in {year}: {min(mins):.1f}°C to {max(maxs):.1f}°C")
            print(f"   Days above baseline {percentile}th percentile: {days_above_p95}/{len(days)} ({days_above_p95/len(days)*100:.1f}%)")
            print(f"   Days above user threshold: {days_above_user}/{len(days)} ({days_above_user/len(days)*100:.1f}%)")
        
        print("\n💡 Interpretation:")
        print(f"   • The {percentile}th percentile represents climatologically extreme temperatures")
        print(f"   • In a stable climate, ~{100-percentile}% of days should exceed this threshold")
        print(f"   • Warming trends show if extreme temperatures are becoming more common")
        print(f"   • Spatial variation shows how temperature extremes vary across your ROI")
            
    except Exception as e:
        print(f"❌ Error in trend analysis: {str(e)}")
        import traceback
        print(f"   Details: {traceback.format_exc()}")

print("✅ Temperature trend analysis function defined")
print("   📈 Long-term trends (2003-2020)")
print("   📊 Daily distribution analysis")
print("   🌡️ Climatological threshold visualization")

✅ Temperature trend analysis function defined
   📈 Long-term trends (2003-2020)
   📊 Daily distribution analysis
   🌡️ Climatological threshold visualization


In [7]:
def run_heat_analysis():
    """
    Main function to calculate extreme heat days with daily air temperature data
    """
    global current_result
    
    with status_output:
        clear_output(wait=True)
        
        # Check if ROI has been set
        if analysis_geom is None:
            print("❌ Please set an ROI first!")
            print("1. Select a test area and click 'Set Test Area'")
            print("2. OR draw an area on the ROI map, then click 'Set ROI from Drawing'")
            print("3. OR upload a reference raster file")
            return
        
        # Get parameters
        year = year_selector.value
        temp_variable = temp_variable_selector.value
        mode = analysis_mode.value
        baseline_period_str = baseline_period.value
        temp_abs = temp_abs_slider.value
        percentile = percentile_slider.value
        
        # Parse baseline period
        baseline_start, baseline_end = parse_baseline_period(baseline_period_str)
        
        # Get ROI area for reporting
        try:
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            area_text = f"({area_km2:.1f} km²)"
        except:
            area_text = ""
        
        # Get temperature variable display name
        temp_var_name = [opt[0] for opt in temp_variable_selector.options if opt[1] == temp_variable][0]
        
        print(f"🔄 Calculating Extreme Heat Days using Daily Air Temperature {area_text}")
        print(f"   Analysis mode: {mode}")
        print(f"   Analysis year: {year}")
        print(f"   Temperature variable: {temp_var_name}")
        print(f"   Baseline period: {baseline_start} to {baseline_end}")
        print(f"   Temperature absolute threshold: {temp_abs}°C")
        print(f"   Temperature relative threshold: {percentile}th percentile")
        print(f"   Resolution: {target_scale}m")
        
        if mode == 'Simplified (Fast)':
            print(f"   Formula: Heat_Day = 1 if T_max > max({temp_abs}°C, overall_p{percentile})")
            print(f"   ⚠️  Simplified mode: No seasonal acclimatization adjustment")
        else:
            print(f"   Formula: Heat_Day = 1 if T_max > max({temp_abs}°C, seasonal_p{percentile})")
            print(f"   ✅ Full mode: Accounts for local acclimatization (±15 day seasonal windows)")
        
        try:
            # Choose calculation method based on mode
            if mode == 'Simplified (Fast)':
                print("   🚀 Using simplified calculation (overall percentile threshold)...")
                
                if temp_variable == 'tmax':
                    extreme_heat_result = calculate_simple_extreme_heat_days_daily(
                        region_geom=analysis_geom,
                        year=year,
                        temp_abs_threshold=temp_abs,
                        percentile_threshold=percentile,
                        baseline_start=baseline_start,
                        baseline_end=baseline_end
                    )
                else:  # tmin
                    print("   ⚠️  Using Tmin for extreme heat analysis is unusual")
                    extreme_heat_result = calculate_simple_extreme_heat_days_daily(
                        region_geom=analysis_geom,
                        year=year,
                        temp_abs_threshold=temp_abs,
                        percentile_threshold=percentile,
                        baseline_start=baseline_start,
                        baseline_end=baseline_end
                    )
            else:
                print("   🔬 Using full calculation (local acclimatization analysis)...")
                print("   This accounts for seasonal heat adaptation and local climate")
                extreme_heat_result = calculate_extreme_heat_days_daily(
                    region_geom=analysis_geom,
                    year=year,
                    temp_abs_threshold=temp_abs,
                    percentile_threshold=percentile,
                    baseline_start=baseline_start,
                    baseline_end=baseline_end,
                    window_days=15  # ±15 day window for proper acclimatization analysis
                )
            
            print(f"   Aligning to target grid ({target_scale}m resolution)...")
            # Align to target grid
            aligned_result = align_to_target_grid(
                extreme_heat_result, 
                analysis_geom, 
                target_projection or 'EPSG:4326',
                target_scale
            )
            
            # Convert to float for export compatibility
            current_result = aligned_result.toFloat().rename('extreme_heat_days')
            
            print(f"   Displaying results on map...")
            # Display on map with memory-efficient visualization
            min_val, max_val = display_on_map(
                m, 
                current_result, 
                analysis_geom, 
                f'Heat Days {year} ({temp_variable})',
                show_region=True
            )
            
            # Enable export button
            export_button.disabled = False
            
            print(f"✅ Analysis complete!")
            print(f"   Range: {min_val:.0f} to {max_val:.0f} extreme heat days per year")
            print(f"   Grid: {target_scale}m resolution (Daily Air Temp 1km)")
            print(f"   ROI area: ~{area_km2:.1f} km²")
            print(f"   Mode: {mode}")
            if mode == 'Simplified (Fast)':
                print(f"   Methodology: Overall percentile (no seasonal adjustment)")
            else:
                print(f"   Methodology: Local acclimatization (±15 day seasonal windows)")
            print(f"   Temperature variable: {temp_var_name}")
            print(f"   Ready for export to GeoTIFF")
            
            # Additional analysis for uniform results
            if min_val == max_val:
                print(f"\n⚠️  All pixels show identical values ({min_val} days)")
                print("   This suggests uniform temperature data or calculation issue")
                print("   💡 Try the '📊 Analyze Year Data' button for detailed diagnostics")
            else:
                print(f"\n✅ Spatial variation detected: {min_val} to {max_val} heat days")
            
        except Exception as e:
            print(f"❌ Error in analysis: {str(e)}")
            import traceback
            print(f"   Details: {traceback.format_exc()}")
            
            if "memory limit" in str(e).lower() or "computation timed out" in str(e).lower():
                print("\n💡 Memory optimization suggestions:")
                print("• Try 'Simplified (Fast)' mode")
                print("• Use shorter baseline period (2010-2019)")
                print("• Use smaller ROI area")
                print("• Try 2km or 5km resolution for large areas")

def export_results():
    """
    Export results to Google Drive
    """
    global export_task
    
    with status_output:
        if current_result is None:
            print("⚠️ Please run analysis first before exporting")
            return
            
        if analysis_geom is None:
            print("⚠️ No ROI defined for export")
            return
        
        # Get parameters for filename
        year = year_selector.value
        temp_variable = temp_variable_selector.value
        mode = analysis_mode.value
        temp_abs = temp_abs_slider.value
        percentile = percentile_slider.value
        
        # Create descriptive filename
        mode_suffix = 'simple' if 'Simplified' in mode else 'acclimatized'
        try:
            area_km2 = int(analysis_geom.area().divide(1000000).getInfo())
            filename = f'daily_heat_days_{year}_{temp_variable}_{mode_suffix}_abs{int(temp_abs)}_p{percentile}_{area_km2}km2_{target_scale}m'
        except:
            filename = f'daily_heat_days_{year}_{temp_variable}_{mode_suffix}_abs{int(temp_abs)}_p{percentile}_{target_scale}m'
        
        print(f"🔄 Exporting to Google Drive...")
        print(f"   Filename: {filename}.tif")
        print(f"   Temperature variable: {temp_variable}")
        print(f"   Mode: {mode}")
        print(f"   Resolution: {target_scale}m")
        print(f"   Projection: {target_projection}")
        print(f"   Format: GeoTIFF")
        
        try:
            export_task = export_to_drive(
                current_result,
                analysis_geom,
                filename,
                target_projection,
                target_scale
            )
            
            print(f"✅ Export task started successfully!")
            print(f"   Task ID: {export_task.id}")
            print(f"   Check Google Earth Engine Tasks tab for progress")
            print(f"   File will appear in your Google Drive/GEE_Exports folder")
            
        except Exception as e:
            print(f"❌ Export failed: {str(e)}")

def analyze_year_temperature_data():
    """
    Comprehensive analysis of temperature data for the full year
    Shows temporal variation and helps identify issues with heat day calculations
    """
    global analysis_geom
    
    if analysis_geom is None:
        print("❌ Please set an ROI first!")
        return
        
    print("📊 Analyzing full year temperature data for your ROI...")
    year = year_selector.value
    temp_variable = temp_variable_selector.value
    temp_abs = temp_abs_slider.value
    percentile = percentile_slider.value
    
    try:
        # Get full year temperature collection
        year_collection = get_temperature_collection(
            analysis_geom, f'{year}-01-01', f'{year}-12-31', temp_variable
        )
        
        count = year_collection.size().getInfo()
        print(f"   Found {count} {temp_variable} images for {year}")
        
        if count == 0:
            print("   ❌ No temperature data found!")
            return
        
        # Sample temperature statistics across the year
        print("   Computing temperature statistics across ROI...")
        
        # Get seasonal samples (4 seasons)
        seasons = [
            ('Summer (Dec-Feb)', f'{year}-12-01', f'{year}-12-31'),  # Southern Hemisphere summer
            ('Autumn (Mar-May)', f'{year}-03-01', f'{year}-05-31'),
            ('Winter (Jun-Aug)', f'{year}-06-01', f'{year}-08-31'),
            ('Spring (Sep-Nov)', f'{year}-09-01', f'{year}-11-30')
        ]
        
        seasonal_stats = []
        for season_name, start_date, end_date in seasons:
            seasonal_collection = year_collection.filterDate(start_date, end_date)
            seasonal_count = seasonal_collection.size().getInfo()
            
            if seasonal_count > 0:
                # Calculate statistics for this season
                seasonal_mean = seasonal_collection.mean()
                stats = seasonal_mean.reduceRegion(
                    reducer=ee.Reducer.minMax().combine(ee.Reducer.mean(), sharedInputs=True),
                    geometry=analysis_geom,
                    scale=2000,
                    maxPixels=1e6,
                    bestEffort=True
                ).getInfo()
                
                temp_min = stats.get('temperature_min', 'N/A')
                temp_max = stats.get('temperature_max', 'N/A')
                temp_mean = stats.get('temperature_mean', 'N/A')
                
                if isinstance(temp_min, (int, float)) and isinstance(temp_max, (int, float)):
                    seasonal_stats.append(f"   {season_name}: {temp_min:.1f}°C to {temp_max:.1f}°C (avg {temp_mean:.1f}°C)")
        
        if seasonal_stats:
            print("   Seasonal temperature ranges:")
            for stat in seasonal_stats:
                print(stat)
        
        # Test threshold calculations
        print(f"\n   Testing thresholds (T_abs={temp_abs}°C, {percentile}th percentile):")
        
        # Get baseline for percentile calculation
        baseline_start, baseline_end = parse_baseline_period(baseline_period.value)
        baseline_collection = get_temperature_collection(
            analysis_geom, baseline_start, baseline_end, temp_variable
        )
        
        baseline_count = baseline_collection.size().getInfo()
        print(f"   Baseline period has {baseline_count} images")
        
        if baseline_count > 0:
            # Calculate percentile threshold
            temp_percentile_img = baseline_collection.select('temperature').reduce(
                ee.Reducer.percentile([percentile])
            ).clip(analysis_geom)
            
            percentile_stats = temp_percentile_img.reduceRegion(
                reducer=ee.Reducer.minMax().combine(ee.Reducer.mean(), sharedInputs=True),
                geometry=analysis_geom,
                scale=2000,
                maxPixels=1e6,
                bestEffort=True
            ).getInfo()
            
            p_min = percentile_stats.get(f'temperature_p{percentile}_min', 'N/A')
            p_max = percentile_stats.get(f'temperature_p{percentile}_max', 'N/A')
            p_mean = percentile_stats.get(f'temperature_p{percentile}_mean', 'N/A')
            
            if isinstance(p_min, (int, float)) and isinstance(p_max, (int, float)):
                print(f"   {percentile}th percentile threshold: {p_min:.1f}°C to {p_max:.1f}°C (avg {p_mean:.1f}°C)")
                
                # Final threshold is max(T_abs, percentile)
                final_thresh_min = max(temp_abs, p_min)
                final_thresh_max = max(temp_abs, p_max)
                print(f"   Final threshold: {final_thresh_min:.1f}°C to {final_thresh_max:.1f}°C")
                
                # Estimate expected heat days
                if seasonal_stats:
                    # Check how many seasons exceed thresholds
                    print(f"\n   🔍 Expected heat days analysis:")
                    print(f"   For a tropical location like Salvador, temperatures are often 28-35°C year-round")
                    print(f"   With threshold {final_thresh_min:.1f}-{final_thresh_max:.1f}°C:")
                    print(f"   - If daily temps frequently exceed threshold → Expect 50-300+ heat days")
                    print(f"   - If only 4 heat days → Threshold may be too high or calculation issue")
                    
            # Check for spatial variation in temperature data
            print(f"\n   🗺️ Checking for spatial temperature variation:")
            sample_img = year_collection.first()
            spatial_stats = sample_img.select('temperature').reduceRegion(
                reducer=ee.Reducer.minMax().combine(ee.Reducer.stdDev(), sharedInputs=True),
                geometry=analysis_geom,
                scale=1000,
                maxPixels=1e6,
                bestEffort=True
            ).getInfo()
            
            spatial_min = spatial_stats.get('temperature_min', 'N/A')
            spatial_max = spatial_stats.get('temperature_max', 'N/A')
            spatial_std = spatial_stats.get('temperature_stdDev', 'N/A')
            
            if isinstance(spatial_std, (int, float)):
                print(f"   Single day temperature variation: {spatial_min:.1f}°C to {spatial_max:.1f}°C")
                print(f"   Standard deviation: {spatial_std:.2f}°C")
                
                if spatial_std < 0.1:
                    print("   ⚠️  Very low spatial variation - this could explain uniform heat day results")
                    print("   💡 Temperature data might be spatially smoothed or have resolution issues")
                else:
                    print("   ✅ Good spatial variation - should produce varied heat day counts")
        
        print(f"\n📊 Analysis complete!")
        print(f"💡 Expected output: Map showing heat days per pixel (0-365 range)")
        print(f"🎯 Each pixel value = count of days in {year} where temperature > threshold")
        print(f"🌈 Color gradient should show spatial variation in heat exposure")
            
    except Exception as e:
        print(f"❌ Error in year analysis: {str(e)}")
        import traceback
        print(f"   Details: {traceback.format_exc()}")

def test_temperature_values():
    """
    Quick diagnostic to check actual temperature values in the data
    """
    global analysis_geom
    
    if analysis_geom is None:
        print("❌ Please set an ROI first!")
        return
        
    print("🔍 Testing temperature values in current ROI...")
    
    # Get appropriate collection for current ROI
    collection_id = get_region_collection(analysis_geom)
    collection = ee.ImageCollection(collection_id)
    
    # Get a few days of 2020 tmax data
    test_collection = collection.filterDate('2020-01-01', '2020-01-10') \
                               .filterBounds(analysis_geom) \
                               .filter(ee.Filter.eq('prop_type', 'tmax'))
    
    count = test_collection.size().getInfo()
    print(f"   Found {count} tmax images in current ROI")
    
    if count > 0:
        # Get first image
        first_image = test_collection.first().select('b1')
        
        # Test raw values (before scaling)
        try:
            raw_stats = first_image.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=analysis_geom,
                scale=2000,  # Use larger scale for speed
                maxPixels=1e6,
                bestEffort=True
            ).getInfo()
            
            raw_min = raw_stats.get('b1_min', 'N/A')
            raw_max = raw_stats.get('b1_max', 'N/A')
            print(f"   Raw values: {raw_min} to {raw_max}")
            
            if isinstance(raw_min, (int, float)) and isinstance(raw_max, (int, float)):
                # Test current scaling (divide by 10)
                scaled_min = raw_min / 10
                scaled_max = raw_max / 10
                print(f"   Current scaling (÷10): {scaled_min:.1f}°C to {scaled_max:.1f}°C")
                
                # Check if values seem reasonable
                if scaled_min < 10 or scaled_max > 50:
                    print("   ⚠️  Temperature values seem unrealistic")
                    print("   Expected range for most locations: ~15-40°C")
                    print("   💡 This might explain why you're getting only 4 heat days")
                else:
                    print("   ✅ Temperature values look reasonable")
                    
                    # Test with 25°C threshold
                    if scaled_max > 25:
                        expected_days = "many" if scaled_min > 25 else "some"
                        print(f"   💡 With max temp {scaled_max:.1f}°C, expect {expected_days} days > 25°C")
                        print("   ⚠️  But this is only one day - use '📊 Analyze Year Data' for full picture")
                    
        except Exception as e:
            print(f"   ❌ Error testing temperature values: {e}")
    else:
        print("   ❌ No tmax data found for test period")

# Connect event handlers
def on_calculate_click(b):
    run_heat_analysis()

def on_export_click(b):
    export_results()

def on_test_temp_click(b):
    with status_output:
        clear_output(wait=True)
        test_temperature_values()

def on_analyze_year_click(b):
    with status_output:
        clear_output(wait=True)
        analyze_year_temperature_data()

calculate_button.on_click(on_calculate_click)
export_button.on_click(on_export_click)

# Add diagnostic buttons
test_temp_button = widgets.Button(
    description='🌡️ Test Temp Values',
    button_style='info',
    layout=widgets.Layout(width='150px', height='35px')
)
test_temp_button.on_click(on_test_temp_click)

analyze_year_button = widgets.Button(
    description='📊 Analyze Year Data',
    button_style='warning',
    layout=widgets.Layout(width='150px', height='35px')
)
analyze_year_button.on_click(on_analyze_year_click)

# Add trend analysis button
trend_analysis_button = widgets.Button(
    description="📊 Temperature Trends",
    button_style="success",
    layout=widgets.Layout(width="150px", height="35px")
)
trend_analysis_button.on_click(lambda b: create_temperature_trend_analysis())

print("✅ Analysis functions ready for daily air temperature processing")
print("   with proper local acclimatization methodology")
print("   🔍 Added comprehensive temperature diagnostics")

✅ Analysis functions ready for daily air temperature processing
   with proper local acclimatization methodology
   🔍 Added comprehensive temperature diagnostics


## Display Analysis Interface

In [8]:
# Create control panel layout
controls_row1 = widgets.HBox([
    year_selector,
    analysis_mode,
    baseline_period
])

controls_row2 = widgets.HBox([
    temp_variable_selector
])

controls_row3 = widgets.HBox([
    temp_abs_slider,
    percentile_slider
])

controls_row4 = widgets.HBox([
    calculate_button,
    export_button,
    test_temp_button,  # Quick temperature test
    analyze_year_button,  # Comprehensive year analysis
    trend_analysis_button  # NEW: Temperature trends
])

control_panel = widgets.VBox([
    widgets.HTML("<h3>🌡️ High-Resolution Daily Extreme Heat Days Calculator</h3>"),
    widgets.HTML("<p><b>Calculate extreme heat days using daily air temperature data at 1km resolution</b></p>"),
    widgets.HTML("<div style='background-color: #d1ecf1; padding: 10px; border-radius: 5px; margin: 10px 0;'>" +
                "<strong>📊 Daily Air Temperature Advantages:</strong><br>" +
                "• <strong>True daily resolution:</strong> 365 images/year vs monthly data<br>" +
                "• <strong>Global coverage:</strong> 5 regional datasets (2003-2020)<br>" +
                "• <strong>High quality:</strong> Station + satellite derived at 1km resolution<br>" +
                "• <strong>Reliable scaling:</strong> Simple divide by 10 for Celsius conversion</div>"),
    widgets.HTML("<div style='background-color: #e7f3ff; padding: 10px; border-radius: 5px; margin: 10px 0;'>" +
                "<strong>🔬 Analysis Modes - Local Acclimatization:</strong><br>" +
                "<strong>Simplified (Fast):</strong> Heat_Day = 1 if T_daily_max > max(T_abs, overall_percentile)<br>" +
                "• Uses single percentile across entire baseline period<br>" +
                "• No seasonal adjustment - same threshold year-round<br>" +
                "• Faster processing but less accurate for acclimatization<br><br>" +
                "<strong>Full Percentiles (Slow):</strong> Heat_Day = 1 if T_daily_max > max(T_abs, seasonal_percentile)<br>" +
                "• Uses ±15 day seasonal windows for each calendar day<br>" +
                "• Accounts for local climate acclimatization and seasonal adaptation<br>" +
                "• What locals consider 'unusually hot' varies by season and location</div>"),
    widgets.HTML("<div style='background-color: #fff3cd; padding: 10px; border-radius: 5px; margin: 10px 0;'>" +
                "<strong>⚠️ Important:</strong> Make sure you've set an ROI first using the ROI selection interface above!</div>"),
    widgets.HTML("<div style='background-color: #e8f5e8; padding: 10px; border-radius: 5px; margin: 10px 0;'>" +
                "<strong>📊 NEW: Temperature Trend Analysis</strong><br>" +
                "Click '📊 Temperature Trends' to visualize:<br>" +
                "• <strong>Long-term trends (2003-2020):</strong> How temperatures change over time<br>" +
                "• <strong>Daily distribution:</strong> Temperature variation across your ROI for selected year<br>" +
                "• <strong>Climatological context:</strong> Compare daily temps vs 95th percentile thresholds<br>" +
                "• <strong>Warming detection:</strong> Linear trend analysis showing climate change<br>" +
                "💡 <em>Perfect for understanding if climatological extremes are becoming more frequent</em></div>"),
    widgets.HTML("<div style='background-color: #ffeaa7; padding: 10px; border-radius: 5px; margin: 10px 0;'>" +
                "<strong>🔍 Troubleshooting Uniform Results:</strong><br>" +
                "If you get uniform results (like 4 heat days everywhere) instead of spatial variation:<br>" +
                "1. <strong>Click '📊 Analyze Year Data'</strong> to see full temperature patterns<br>" +
                "2. Check seasonal temperature ranges and thresholds<br>" +
                "3. Verify spatial temperature variation across your ROI<br>" +
                "4. Expected: Color gradient showing 0-365 heat days per pixel<br>" +
                "💡 <em>Each pixel should show count of days above threshold in that year</em></div>"),
    widgets.HTML("<div style='background-color: #d4edda; padding: 10px; border-radius: 5px; margin: 10px 0;'>" +
                "<strong>🌡️ Climatological Approach - Percentiles as Extremes:</strong><br>" +
                "• <strong>90th-95th percentiles represent true climatological extremes</strong><br>" +
                "• These thresholds account for local climate acclimatization<br>" +
                "• 11 heat days in Salvador = ~3% of year above 95th percentile (expected: 5%)<br>" +
                "• Coastal tropical climates have very stable temperatures<br>" +
                "• Use trend analysis to detect if extremes are becoming more frequent<br>" +
                "💡 <em>This is the scientifically correct approach for climate analysis</em></div>"),
    controls_row1,
    controls_row2,
    controls_row3,
    controls_row4,
    status_output
])

# Display the complete interface
display(control_panel)
display(m)

print("\n🎯 High-Resolution Daily Extreme Heat Days Calculator Ready!")
print("\n📋 Workflow:")
print("1. 🎯 Set ROI: Use the ROI selection interface above")
print("2. 📊 Explore Trends: Click '📊 Temperature Trends' for temporal analysis")
print("3. 📊 Analyze Year Data: Click '📊 Analyze Year Data' to understand temperature patterns")
print("4. 🌡️ Select Temperature Variable: Daily Maximum (Tmax) recommended for heat analysis")
print("5. ⚙️ Configure: Adjust thresholds based on your climate zone")
print("6. 🔬 Choose Mode: Full Percentiles for proper local acclimatization analysis")
print("7. 🔥 Calculate: Click 'Calculate Heat Days' to run analysis")
print("8. 👁️ Review: Check results on the map - should show spatial variation as color gradient")
print("9. 📁 Export: Click 'Export to Drive' to download GeoTIFF")

print("\n📊 Expected Output:")
print("• Map showing heat days per pixel (0-365 range)")
print("• Each pixel value = count of days in year where temperature > threshold")
print("• Color gradient showing spatial variation in heat exposure")
print("• NOT uniform values - should vary across space")

print("\n🔍 Analysis Tools:")
print("• '🌡️ Test Temp Values': Quick check of temperature data for one day")
print("• '📊 Analyze Year Data': Comprehensive analysis of full year temperature patterns")
print("• '📊 Temperature Trends': Long-term trends and daily distribution analysis")
print("• Use these to understand your data before running main analysis")

print("\n📊 Daily Air Temperature Dataset Features:")
print("• Temporal Coverage: 2003-2020 (18 years)")
print("• Spatial Resolution: 1km (30 arcsecond)")
print("• Temporal Resolution: Daily (both Tmax and Tmin available)")
print("• Regional Coverage: 5 global regions with seamless coverage")
print("• Data Quality: Station + satellite observations with machine learning")

print("\n🔬 Climatological Approach:")
print("• 90th-95th percentiles = True climatological extremes")
print("• Accounts for local climate and seasonal acclimatization")
print("• 11 heat days in Salvador = scientifically reasonable for stable tropical climate")
print("• Use trend analysis to detect climate change impacts")
print("• Focus on whether extremes are becoming more frequent over time")

Map(center=[20.0, 0.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…


🎯 High-Resolution Daily Extreme Heat Days Calculator Ready!

📋 Workflow:
1. 🎯 Set ROI: Use the ROI selection interface above
2. 📊 Explore Trends: Click '📊 Temperature Trends' for temporal analysis
3. 📊 Analyze Year Data: Click '📊 Analyze Year Data' to understand temperature patterns
4. 🌡️ Select Temperature Variable: Daily Maximum (Tmax) recommended for heat analysis
5. ⚙️ Configure: Adjust thresholds based on your climate zone
6. 🔬 Choose Mode: Full Percentiles for proper local acclimatization analysis
7. 🔥 Calculate: Click 'Calculate Heat Days' to run analysis
8. 👁️ Review: Check results on the map - should show spatial variation as color gradient
9. 📁 Export: Click 'Export to Drive' to download GeoTIFF

📊 Expected Output:
• Map showing heat days per pixel (0-365 range)
• Each pixel value = count of days in year where temperature > threshold
• Color gradient showing spatial variation in heat exposure
• NOT uniform values - should vary across space

🔍 Analysis Tools:
• '🌡️ Test Temp 

## Results Management

In [55]:
# Results Display and Management
print("📊 Analysis Results Management")
print("Use this cell to check current analysis status and manage results")

if 'current_result' in globals() and current_result is not None:
    print("\n✅ Analysis result available")
    
    if 'analysis_geom' in globals() and analysis_geom is not None:
        try:
            # Show ROI info
            area_km2 = analysis_geom.area().divide(1000000).getInfo()
            print(f"   ROI area: {area_km2:.1f} km²")
            
            # Quick data check
            test_stats = current_result.reduceRegion(
                reducer=ee.Reducer.count().combine(ee.Reducer.minMax(), sharedInputs=True),
                geometry=analysis_geom,
                scale=2000,
                maxPixels=1e6,
                bestEffort=True
            ).getInfo()
            
            pixel_count = test_stats.get('extreme_heat_days_count', 0)
            min_val = test_stats.get('extreme_heat_days_min', None)
            max_val = test_stats.get('extreme_heat_days_max', None)
            
            print(f"   Data coverage: {pixel_count} pixels")
            print(f"   Value range: {min_val} to {max_val} heat days")
            
            if pixel_count == 0:
                print("   ⚠️ No data found - check ROI and parameter settings")
                print("   💡 Try different temperature variable or thresholds")
            elif min_val == max_val == 0:
                print("   ⚠️ All pixels show 0 heat days - thresholds may be too high")
                print("   💡 Try lower absolute threshold or percentile (see climate guidance)")
            else:
                print("   ✅ Ready for export!")
                
        except Exception as e:
            print(f"   Status check failed: {e}")
    else:
        print("   ⚠️ No ROI defined")
        
else:
    print("\n⭕ No analysis result found")
    print("   Please run the analysis using the interface above")

print("\n🗺️ Map Layers:")
print("• Blue boundary = Current ROI")  
print("• Heat colors = Extreme heat days per year (Daily Air Temp 1km)")
print("• Use map layer controls to toggle visibility")

print("\n🌡️ Daily Air Temperature Dataset Info:")
print("• Spatial Resolution: 1km (30 arcseconds)")
print("• Temporal Coverage: 2003-2020 (18 years)")
print("• Temporal Resolution: Daily (Tmax and Tmin)")
print("• Regional Coverage: 5 global collections with seamless boundaries")
print("• Data Source: Station observations + satellite data with machine learning")
print("• Scaling: Raw values ÷ 10 = Temperature in Celsius")

print("\n💡 Troubleshooting Zero Heat Days:")
print("• Check if thresholds are appropriate for your climate zone")
print("• Tropical: try 30°C + 80th percentile")
print("• Temperate: try 35°C + 90th percentile")
print("• Desert: try 40°C + 95th percentile")
print("• Consider using lower percentiles (75th-85th) for tropical areas")

📊 Analysis Results Management
Use this cell to check current analysis status and manage results

⭕ No analysis result found
   Please run the analysis using the interface above

🗺️ Map Layers:
• Blue boundary = Current ROI
• Heat colors = Extreme heat days per year (Daily Air Temp 1km)
• Use map layer controls to toggle visibility

🌡️ Daily Air Temperature Dataset Info:
• Spatial Resolution: 1km (30 arcseconds)
• Temporal Coverage: 2003-2020 (18 years)
• Temporal Resolution: Daily (Tmax and Tmin)
• Regional Coverage: 5 global collections with seamless boundaries
• Data Source: Station observations + satellite data with machine learning
• Scaling: Raw values ÷ 10 = Temperature in Celsius

💡 Troubleshooting Zero Heat Days:
• Check if thresholds are appropriate for your climate zone
• Tropical: try 30°C + 80th percentile
• Temperate: try 35°C + 90th percentile
• Desert: try 40°C + 95th percentile
• Consider using lower percentiles (75th-85th) for tropical areas


In [ ]:
# Create trend analysis button
print("✅ Temperature trend analysis tools ready")
print("   📈 Long-term trends (2003-2020)")
print("   📊 Daily distribution analysis")
print("   🌡️ Climatological threshold visualization")